In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
# import necessary packages

import os
import numpy as np
import pandas as pd
import scipy
from scipy import stats
import ast

import seaborn as sns
from matplotlib import pyplot as plt

from collections import defaultdict

from sklearn.feature_selection import f_classif

In [ ]:
DIR_WSI = "/Users/jinzhou/Desktop/USCAP/data/wsi"
DIR_ANN = "/Users/jinzhou/Desktop/USCAP/data/ann_geojsons"
DIR_SAVE = "/Users/jinzhou/Desktop/USCAP/data"

## Process thickness.csv

In [ ]:
PATH_STAT = os.path.join(DIR_SAVE, "thickness.csv")

In [ ]:
# read thickness analysis
def read_df_from_csv(path_csv):
    df = pd.read_csv(path_csv)
    # original list in path_csv is read as string, for example "[0.2, 0.1, ...]"
    # ast.literal_eval converts string to python literal structures
    df['Thickness_Media_Abs'] = df['Thickness_Media_Abs'].apply(lambda x: ast.literal_eval(x))
    df['Thickness_Intima_Abs'] = df['Thickness_Intima_Abs'].apply(lambda x: ast.literal_eval(x))
    df['Thickness_Wall_Abs'] = df['Thickness_Wall_Abs'].apply(lambda x: ast.literal_eval(x))
    return df

df_thick = read_df_from_csv(PATH_STAT)

In [ ]:
# A few adjustment to match qupath annotations to labels assignment
# 1. In "11_26609_023_512 L4 TRI", A8 -> A08
# 2. In "11_26609_009_008 L10 TRI", there are two A26, where the second one should be A27
# 3. discard "12_26609_021_507 L03 TRI" from analysis

df_thick.loc[:, 'Artery_ID'] = df_thick.loc[:, 'Artery_ID'].str.split('_').str[0]

df_thick.loc[(df_thick.loc[:, 'WSI_ID'] == '11_26609_009_008 L10 TRI') & 
       (df_thick.loc[:, 'Artery_ID'] == 'A26'), 'Artery_ID'] = ['A26', 'A27']

df_thick.loc[(df_thick.loc[:, 'WSI_ID'] == '11_26609_023_512 L4 TRI') & 
       (df_thick.loc[:, 'Artery_ID'] == 'A8'), 'Artery_ID'] = 'A08'

df_thick = df_thick[df_thick.loc[:, "WSI_ID"]!="12_26609_021_507 L03 TRI"]

In [ ]:
# Set and WSI_Artery_ID, ignoring media and intima index
df_thick.loc[:, 'WSI_Artery_ID'] = df_thick.loc[:, 'WSI_ID'] + '_' \
    + df_thick.loc[:, 'Artery_ID']

In [ ]:
# For arteries with multiple lumen/intima areas, we pick the one with maxium lumen area
df_thick = df_thick.sort_values('Area', ascending=False).drop_duplicates(['WSI_Artery_ID'])

## Process "Labels.xlsx"

In [ ]:
PATH_LABEL = "/Users/jinzhou/Desktop/USCAP/data/raw/labels_updated.xlsx"

In [ ]:
df_label = pd.ExcelFile(PATH_LABEL)

# Only read the first sheet of the excel file
sheet_name = df_label.sheet_names[0]
df_label = df_label.parse(sheet_name, skiprows=1) # skip the first row
# Use Artery_ID as row index and WSI_ID as column name
df_label = df_label.rename(columns = {'Unnamed: 0':'Artery_ID'}).set_index("Artery_ID")

In [ ]:
# replace labels of strings to integers.
df_label = df_label.replace({"without arteriosclerosis": 0, 
                             "mild arteriosclerosis": 1, 
                             "mild hyalinosis": 1, 
                             "moderate arteriosclerosis": 2,
                             "severe arteriosclerosis": 3, 
                             "-": np.nan, " - ": np.nan})


## Clip and Normalize Thickness

In [ ]:
def plot_hist_w_two_list(thickness_outer, thickness_inner, xlabel, excludes, path_to_svae):
    for val in excludes:
        thickness_outer = [x for x in thickness_outer if x!=val]
        thickness_inner = [x for x in thickness_inner if x!=val]
    bins = np.linspace(0, np.max(thickness_outer+thickness_inner), 100)
    plt.figure(figsize=(10, 5))    
    plt.hist(thickness_outer, bins=bins, alpha=0.5, label="Media") 
    plt.hist(thickness_inner, bins=bins, alpha=0.5, label="Intima")
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel(xlabel, fontsize=20)
    plt.ylabel("Frequency", fontsize=20)
    plt.legend(loc='upper right', fontsize=20)
    if path_to_svae:
        plt.savefig(path_to_svae)
    plt.show()
    

def clip_normalize(thick_media, thick_intima, thick_wall, plot_hist=False):
    assert len(thick_media) == len(thick_intima) == len(thick_wall) == 360
    
    # visualize hist before processing
    if plot_hist:
        plot_hist_w_two_list(thick_media, thick_intima, "Thickness", [-1], None)
    # Clip by 0.05 * median value of  thick_wall
    # remove all -1s, -1 means not intersection/thickness was found at this sample    
    clip_th = 0.05*np.median([x for x in thick_wall if x!= -1])
    thick_media_norm = [-1]*len(thick_media)
    thick_intima_norm = [-1]*len(thick_intima)
    for i in range(len(thick_wall)):
        if thick_media[i] < clip_th or thick_intima[i] < clip_th: 
            # if either media or intima thickness is below threshold, discard both
            continue
        else:
            thick_media_norm[i] = thick_media[i] / thick_wall[i]
            thick_intima_norm[i] = thick_intima[i] / thick_wall[i]
    thick_media_norm = [x for x in thick_media_norm if x!=-1]
    thick_intima_norm = [x for x in thick_intima_norm if x!=-1]
    if plot_hist:
        plot_hist_w_two_list(thick_media_norm, thick_intima_norm, 
                             "Clipped and Normalizied Thickness", [], plot_hist)
    return thick_media_norm, thick_intima_norm

def get_features(dict_features, l_thick, prefix): 
    arr_thick = np.array(sorted(l_thick, reverse=True))
    # average of the top 5% value
    dict_features[prefix+" Average of Top 5%"] = np.mean(arr_thick[:len(arr_thick)//20])
    dict_features[prefix+" Skewness"] = scipy.stats.skew(arr_thick)
    dict_features[prefix+" Power"] = scipy.sum(arr_thick**2)/arr_thick.size

In [ ]:
df_features_label = pd.DataFrame(columns = ["WSI_Artery_ID", "Media Average of Top 5%", "Media Skewness", "Media Power", 
                                             "Intima Average of Top 5%", "Intima Skewness", "Intima Power",
                                             "Label"])

for index, row in df_thick.iterrows():
    thick_media = row["Thickness_Media_Abs"]
    thick_intima = row["Thickness_Intima_Abs"]
    thick_wall = row["Thickness_Wall_Abs"]
    path_save_hist = os.path.join(DIR_SAVE, row["WSI_ID"], row["Artery_ID"]+"_hist.png")
    
    # clip and normalize
    thick_media, thick_intima = clip_normalize(thick_media = row["Thickness_Media_Abs"], 
                                               thick_intima = row["Thickness_Intima_Abs"], 
                                               thick_wall = row["Thickness_Wall_Abs"], plot_hist=None)
    # feature extraction
    row_features_label = {}
    features_media = get_features(row_features_label, thick_media, "Media")
    features_intima = get_features(row_features_label, thick_intima, "Intima")
    row_features_label["WSI_Artery_ID"] = row["WSI_Artery_ID"]
    row_features_label["Label"] = df_label.loc[row["Artery_ID"], row["WSI_ID"]]
    df_features_label = df_features_label.append(row_features_label, ignore_index=True)

In [ ]:
df_features_label.head()

## HeatMap, Boxtplot and Kendall Tau Analysis

In [ ]:
def heatmap_features(df_features_label, feature_name):
    xticklabels = df_features_label.loc[:, "WSI_Artery_ID"].values
    features = df_features_label.loc[:, feature_name].values
    labels = df_features_label.loc[:, "Label"].values

    fig = plt.figure(constrained_layout=True, figsize=(24, 4))
    axs = fig.subplot_mosaic([['TopLeft', 'Right'],['BottomLeft', 'Right']],
                              gridspec_kw={'width_ratios':[5, 1]})
    
    # heatmap
    idx_sort = features.argsort()
    sns.heatmap(features[idx_sort].reshape(1, -1), cbar=True, 
                cbar_kws = dict(use_gridspec=False,location="bottom"), 
                ax=axs['TopLeft'])
    axs['TopLeft'].set_xticks([])
    axs['TopLeft'].set_yticks([])
    axs['TopLeft'].set_ylabel("Feature", fontsize=15)
    sns.heatmap(labels[idx_sort].reshape(1, -1), cmap='gray', cbar=True, 
                cbar_kws = dict(use_gridspec=False,location="bottom"), 
#                 xticklabels =xticklabels[idx_sort] ,
                ax=axs['BottomLeft'])
    axs['BottomLeft'].set_xticks([])
    axs['BottomLeft'].set_yticks([])
    axs['BottomLeft'].set_ylabel("Score", fontsize=15)
    idx_label_0 = labels==0
    idx_label_1 = labels==1
    idx_label_2 = labels==2
    idx_label_3 = labels==3
    
    # box plot
    features_label_0 = features[idx_label_0]
    features_label_1 = features[idx_label_1]
    features_label_2 = features[idx_label_2]
    features_label_3 = features[idx_label_3]
    axs['Right'].boxplot([features_label_0, features_label_1, features_label_2, features_label_3])
    axs['Right'].set_xticklabels([0,1,2,3], fontsize=12)  
    axs['Right'].tick_params(axis="y", labelsize=12)
    axs['Right'].set_xlabel("Arteriosclerosis Score", fontsize=15)
    axs['Right'].set_ylabel("Feature", fontsize=15)
    
    # Kendall Tau
    rho, p_val = scipy.stats.kendalltau(features, labels)
    plt.show()
    print('Feature: {}; kendalltau: rho = {:.2f}, p = {:.6f}'.format(feature_name, rho, p_val))
    
for col in ["Media Average of Top 5%", "Media Skewness", "Media Power", 
            "Intima Average of Top 5%", "Intima Skewness", "Intima Power"]:
    heatmap_features(df_features_label, col)

In [ ]:
df_features_label

In [ ]:
path_to_save = os.path.join(DIR_SAVE, "features_label.csv")
df_features_label.to_csv(path_to_save, index=False)  
    